In [1]:
#imports and setup
import numpy as np
import cv2 as cv
import os

In [2]:
'''loading video names into a list for easier access with absolute path,
video names can be access by calling the index of the video_lst and will contain 
the current absolute path, asumes that the video folder is in the same folder as this file'''

video_path = "videos"
full_path = os.path.abspath(video_path)

video_lst = []
v_dir = os.listdir(full_path)
for name in v_dir:
    video_lst.append(full_path+'/'+name)
print(video_lst[0]) 

/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/computer_vision_prototype-main/videos/video_026.mp4


In [3]:
#loading video into memory
cap = cv.VideoCapture(video_lst[0])

In [ ]:
'''Playing one video file'''

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    '''cv_waitkey() defines miliseconds between frames, higher values play the video slower
    ord('q') defines a key to exit the video'''
    cv.imshow('frame',frame)
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
cap.release() #after release from memory the video needs to be recaptured
cv.destroyAllWindows()

In [ ]:
'''Testing background substraction with morphological noise reduction
    opencv has KNN and MOG2 BGS build in'''

cap = cv.VideoCapture(video_lst[0])
backSub = cv.createBackgroundSubtractorKNN()
#backSub = cv.createBackgroundSubtractorMOG2()

#kernel for morphological noise reduction
#cv.MORPH_ELLIPSE
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3));
while cap.isOpened():
    ret, frame = cap.read()
    if frame is None:
        break
    
    fgMask = backSub.apply(frame)
    fgMask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, kernel);

    #cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()

In [4]:
# Daria - Euclidean Distance Tracker

import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


In [ ]:
# tracker types in opencv -> needs different version of opencv to run
tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']


tracker = tracker_types[1]

if tracker == 'BOOSTING':
    tracker = cv.legacy.TrackerBoosting_create()
if tracker == 'MIL':
    tracker = cv.TrackerMIL_create() 
if tracker == 'KCF':
    tracker = cv.TrackerKCF_create() 
if tracker == 'TLD':
    tracker = cv.legacy.TrackerTLD_create() 
if tracker == 'MEDIANFLOW':
    tracker = cv.legacy.TrackerMedianFlow_create() 
# if tracker_type == 'GOTURN':
#     tracker = cv.TrackerGOTURN_create()
if tracker == 'MOSSE':
    tracker = cv.legacy.TrackerMOSSE_create()
if tracker == "CSRT":
    tracker = cv.TrackerCSRT_create()

print(f"Current tracker type: {tracker}")

In [5]:
# Utility Functions
# Compute the frame difference
def frame_diff(prev_frame, cur_frame, next_frame):
    diff_frames1 = cv.absdiff(next_frame, cur_frame)
    # Absolute difference between current frame and previous frame
    diff_frames2 = cv.absdiff(cur_frame, prev_frame)
    # Return the result of bitwise 'AND' between the above two resultant images
    #gives better result than simple substraction
    return cv.bitwise_and(diff_frames1, diff_frames2)

def get_frame(cap):
    ret, frame = cap.read()
    # Resize the image not needed
    '''frame = cv.resize(frame, None, fx=scaling_factor,
        fy=scaling_factor, interpolation=cv.INTER_AREA)'''
    return frame

#finds and draws contours over pixels if greater than threshold
def construct_contours(frame, contour_threshold):
    frame = cv.dilate(frame, (1,1), iterations=1)
    contours, hierarchy = cv.findContours(frame, cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE)
    '''cv.drawContours(frame_th, contours=contours, contourIdx=-1, 
                     color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)'''
    detections=[]
    for contour in contours:
      if cv.contourArea(contour) < contour_threshold:
        continue
      (x, y, w, h) = cv.boundingRect(contour)
      #cv.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
      detections.append([x,y,w,h])
    
    return detections

    #applying blur will possibly extend with other option for preperation
def prepare_frames(frames,kernel):
    result = []
    for frame in frames:
        result.append(cv.GaussianBlur(frame, kernel, 0))
    return result[0], result[1], result[2]

In [7]:
# Trying out tracker:


'''Testing Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding'''

#video to capture

video_lst = sorted(video_lst)
video = video_lst[5]
print(video)

kernel = np.ones((7,7)) #kerne for dilation
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
THRESHOLD = 20 #when to detect difference
VALUE = 255 #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = EuclideanDistTracker()


cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
#next_frame = get_frame(cap)
ret, next_frame = cap.read()

print(type(next_frame))

centerpoints = []

trackerrInit = False


# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    #applying preprocessing
    if type(next_frame) is None:
         break
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow('frame diff', frame_difference)
    #applying dilation
    frame_difference = cv.dilate(frame_difference, kernel)
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, 30)
    # detections -> bb coordinates
    
    #cv.imshow("Difference", frame_difference)
    cv.imshow("After Threshold", frame_th)
    #cv.imshow("Frame", cur_frame)

    # if not trackerrInit:
    #     # Initialize tracker with first frame and bounding box
    #     #tracker.init(cur_frame,)
    #     trackerrInit = True


    boxes_ids = tracker.update(detections)

    #boxes_ids = tracker.update(cur_frame)


    # detection
    for box_id in boxes_ids:
        
        x,y,w,h,id = box_id

        
        # get coordinates of center point
        cx = int((x + x + w) / 2)  
        cy = int((y + y + h) / 2)
        
        centerpoints.append((cx, cy))


        cv.putText(cur_frame, "frame:" + str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 218, 0))
        cv.putText(cur_frame,  str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv.rectangle(cur_frame, (x,y),(x+w, y+h), (0,255,0), 2)
        
#        for point in centerpoints:
 #           cv.circle(cur_frame, point, 3, (0,0,255), -1 )
        #cv.imshow('frame', cur_frame)

    # show circle around centerpoints
    for point in centerpoints:
        cv.circle(cur_frame, point, 3, (0,0,255), 0 )
    cv.imshow('frame', cur_frame)


    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    # next_frame = get_frame(cap)
    ret, next_frame = cap.read()
    
    if not ret:
         break

    if cv.waitKey(25) & 0xFF == ord('q'):
        break    
cap.release()
cv.waitKey(1)

cv.destroyAllWindows()
for i in range (1,5):
    cv.waitKey(1)
#cap.release()



/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/computer_vision_prototype-main/videos/video_005.mp4
<class 'numpy.ndarray'>
{0: (129, 140)}
{0: (128, 138)}
{0: (129, 139)}
{0: (130, 139)}
{0: (132, 140)}
{0: (129, 140)}
{0: (132, 139)}
{0: (134, 138)}
{0: (132, 138)}
{0: (131, 137)}
{0: (132, 137)}
{0: (129, 139)}
{0: (131, 138)}
{0: (129, 139)}
{0: (130, 139)}
{0: (128, 138)}
{0: (129, 137)}
{0: (129, 140)}
{0: (129, 140)}
{0: (129, 139)}
{0: (129, 139)}
{0: (129, 140)}
{0: (129, 140)}
{0: (129, 140), 1: (225, 272)}
{1: (225, 271), 0: (129, 140)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 270), 0: (129, 139)}
{1: (225, 270), 0: (130, 139)}
{1: (225, 271), 0: (130, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (130, 138)}
{1: (225, 271), 0: (130, 138)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 138)}
{1: (225, 271), 0: (129, 138)}
{1: (225, 271), 0: (132, 139)}
{1: (225, 271), 0: (132

In [9]:
#With the option to save videos
# Trying out tracker:
'''Testing Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding'''

#video to capture

video_lst = sorted(video_lst)
video = video_lst[5]
print(video)

kernel = np.ones((7,7)) #kerne for dilation
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7)) #kernel for morphology
THRESHOLD = 20 #when to detect difference
VALUE = 255 #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = EuclideanDistTracker()


cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
#next_frame = get_frame(cap)
ret, next_frame = cap.read()

print(type(next_frame))

centerpoints = []

trackerrInit = False

'''Check if directory exists, if not, create it'''
import os

# You should change 'test' to your preferred folder.
dir = ("outputs")
check = os.path.isdir(dir)

# If folder doesn't exist, then create it.
if not check:
    os.makedirs(dir)
    print("created folder : ", dir)

else:
    print(dir, "folder already exists.")

file_name = video.split("/")[-1].split(".")
file_name[0] = file_name[0]+"_out"
output_name = ".".join(file_name)
full_output_path = f"{dir}/{output_name}"

result = cv.VideoWriter(full_output_path, cv.VideoWriter_fourcc(*'XVID'), 20, (640, 512))


# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    #applying preprocessing
    if type(next_frame) is None:
         break
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow('frame diff', frame_difference)
    #applying dilation
    frame_difference = cv.dilate(frame_difference, kernel)
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, 30)
    # detections -> bb coordinates
    
    #cv.imshow("Difference", frame_difference)
    cv.imshow("After Threshold", frame_th)
    #cv.imshow("Frame", cur_frame)

    # if not trackerrInit:
    #     # Initialize tracker with first frame and bounding box
    #     #tracker.init(cur_frame,)
    #     trackerrInit = True


    boxes_ids = tracker.update(detections)

    #boxes_ids = tracker.update(cur_frame)


    # detection
    for box_id in boxes_ids:
        
        x,y,w,h,id = box_id

        
        # get coordinates of center point
        cx = int((x + x + w) / 2)  
        cy = int((y + y + h) / 2)
        
        centerpoints.append((cx, cy))


        cv.putText(cur_frame, "frame:" + str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 218, 0))
        cv.putText(cur_frame,  str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv.rectangle(cur_frame, (x,y),(x+w, y+h), (0,255,0), 2)
        
#        for point in centerpoints:
 #           cv.circle(cur_frame, point, 3, (0,0,255), -1 )
        #cv.imshow('frame', cur_frame)

    # show circle around centerpoints
    for point in centerpoints:
        cv.circle(cur_frame, point, 1, (0,255,0), 0 )
    cv.imshow('frame', cur_frame)

    result.write(cur_frame)
    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    # next_frame = get_frame(cap)
    ret, next_frame = cap.read()
    
    if not ret:
         break

    if cv.waitKey(25) & 0xFF == ord('q'):
        break    
cap.release()
result.release()
cv.waitKey(1)

cv.destroyAllWindows()
for i in range (1,5):
    cv.waitKey(1)
#cap.release()



/Users/daryashitova/Documents/MA_AI/WS22:23/Computer Vision/project/computer_vision_prototype-main/videos/video_005.mp4
<class 'numpy.ndarray'>
outputs folder already exists.


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


{0: (130, 140)}
{0: (128, 138)}
{0: (129, 138)}
{0: (131, 139)}
{0: (131, 139)}
{0: (130, 140)}
{0: (133, 138)}
{0: (132, 138)}
{0: (130, 138)}
{0: (131, 138)}
{0: (131, 138)}
{0: (129, 139)}
{0: (131, 138)}
{0: (130, 139)}
{0: (130, 138)}
{0: (129, 139)}
{0: (129, 138)}
{0: (129, 140)}
{0: (129, 139)}
{0: (129, 139)}
{0: (130, 141)}
{0: (129, 139)}
{0: (129, 139)}
{0: (129, 139), 1: (225, 272)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 139)}
{1: (225, 271), 0: (129, 138)}
{1: (225, 271), 0: (129, 138)}
{1: (225, 271), 0: (130, 140)}
{1: (225, 271), 0: (130, 140)}
{1: (225, 271), 0: (130, 139)}
{1: (225, 271), 0: (130, 139)}
{1: (225, 271), 0: (130, 138)}
{1: (225, 271), 0: (130, 138)}
{1: (225, 271), 0: (131, 139)}
{1: (225, 271), 0: (131, 139)}
{1: (225, 271), 0: (131, 139)}
{1: (225, 271), 0: (131, 139)}
{1: (225, 271), 0: (131, 138)}
{1: (225, 271), 0: (131, 138)}
{1: (225, 27